# Pytorch Test Youtube Training Models
Notebook for following along with Pytorch model building, using [Pytorch](https://pytorch.org/tutorials/beginner/introyt/trainingyt.html) website tutorial. This notebook will be similar to the previous Pytorch Test notebooks, as the [youtube content](https://www.youtube.com/watch?v=jF43_wj_DCQ) covers similiar works.

### Choices for data

<br>

### Libaries and Modules
Importing the necessary libaries and modules for the notebook.

In [1]:
#Import cell
import matplotlib as mpl
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import math
import numpy as np
import pandas as pd
import pickle as pk
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

from datetime import datetime
from torch.utils.tensorboard import SummaryWriter

print("Imports complete")

Imports complete


<br>

### Importing and preparing data sets
Importing and preparing the data for the models.

In [2]:
#Gather datasets and prepare them for consumption
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

In [3]:
#Importing data sets
training_set = torchvision.datasets.FashionMNIST('./data',
                                                 train=True,
                                                 transform=transform,
                                                 download=True)
validation_set = torchvision.datasets.FashionMNIST('./data',
                                                   train=False,
                                                   transform=transform,
                                                   download=True)
print(f"Training set has {len(training_set)} instances.")
print(f"Training set has {len(validation_set)} instances.")
print("Data sets successfully imported.")

Training set has 60000 instances.
Training set has 10000 instances.
Data sets successfully imported.


In [4]:
#Loader definitions
training_loader = torch.utils.data.DataLoader(training_set,
                                              batch_size=4,
                                              shuffle=True,
                                              num_workers=2)
validation_loader = torch.utils.data.DataLoader(validation_set,
                                                batch_size=4,
                                                shuffle=False,
                                                num_workers=2)
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
        'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')
print("Loaders defined.")

Loaders defined.


In [5]:
#Setting seed value
torch.manual_seed(1247)

<br>

### Class Definitions
<b>Classes:</b><br>
<ul>
    <li>GarmentClassifier - convolutional layered nn for classifying images</li>
</ul>

In [6]:
#Class definition cell
class GarmentClassifier(nn.Module):
    def __init__(self):
        super(GarmentClassifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*4*4, 120)
        self. fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        return None
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*4*4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

print("Classes defined.")

Classes defined.


<br>

### Calculation functions
<b>Functions:</b><br>
<ul>
    <li></li>
</ul>

In [7]:
#Calculation functions cell

print("Calculation functions defined.")

Calculation functions defined.


<br>

### Plotting functions
<b>Functions:</b>
<ul>
    <li><b>matplotlib_imshow</b> - shows an input image</li>
</ul>

In [8]:
#Plotting functions Cell
def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))

print("Plotting functions defined.")

Plotting functions defined.


<br>

### Main code
#### Dataset and DataLoader
These classes encapsulate the process of pulling your data from storage and exposing it to your training loop in batchs.

In [9]:
dataiter = iter(training_loader)
images, labels = dataiter.next()

##### Convert this cell to code to preview images
img_grid = torchvision.utils.make_grid(images)
matplotlib_imshow(img_grid, one_channel=True)
print('  '.join(classes[labels[j]] for j in range(4)))

#### The Model

In [10]:
model = GarmentClassifier()

#### Loss Function
Below is a demonstration of implementing the loss function using cross-entropy loss.

In [11]:
loss_fn = torch.nn.CrossEntropyLoss()
dummy_outputs = torch.rand(4, 10)
dummy_labels = torch.tensor([1, 5, 3, 7])

print(dummy_outputs)
print(dummy_labels)

loss = loss_fn(dummy_outputs, dummy_labels)
print(f'Total loss for this batch: {loss.item()}')

tensor([[0.9723, 0.5379, 0.5921, 0.4921, 0.8621, 0.2427, 0.6276, 0.1297, 0.6423,
         0.8299],
        [0.4719, 0.2032, 0.5063, 0.5815, 0.3624, 0.1268, 0.3638, 0.8573, 0.2414,
         0.3413],
        [0.9080, 0.4103, 0.1342, 0.3986, 0.5414, 0.4161, 0.5911, 0.1580, 0.9634,
         0.0017],
        [0.7062, 0.8295, 0.2613, 0.9366, 0.7074, 0.0130, 0.9735, 0.9418, 0.7061,
         0.3764]])
tensor([1, 5, 3, 7])
Total loss for this batch: 2.3603768348693848


#### Optimizer
Here stochastic gradient descent will be used. Experimenting with the parameters may lead to differences in the final model and the time it takes to converge.

In [12]:
optimizer = torch.optim.SGD(model.parameters(), lr = 0.001,
                            momentum = 0.9)

#### The Training Loop

In [13]:
def train_one_epoch(epoch_inde, tb_writer):
    running_loss = 0.
    last_loss = 0.
    
    for i, data in enumerate(training_loader): 
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i%1000 == 999:
            last_loss = running_loss/1000 #loss per batch
            tb_x = epoch_index*len(training_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.
        
    return last_loss

#### Per-Epoch Activity

<br>